In [2]:
from matplotlib import pyplot
from keras.datasets import cifar10

# load the dataset
(train_x, train_y), (test_x, test_y) = cifar10.load_data()

In [3]:
# Preparing the data
from tensorflow.keras.utils import to_categorical
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

train_x = train_x/ 255
test_x = test_x /255

In [4]:
# Function to split train dataset into sub-training and validation where validation dataset is 1/5 of training set
from sklearn.model_selection import train_test_split
from numpy import random

def train_valid_split(X_data, y_data):
    return train_test_split(X_data, y_data, test_size=0.2, shuffle=True)

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, MaxPool2D
from keras.layers.convolutional import Conv2D
import tensorflow as tf
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator

# Architecture 3 - using dropout, regularizers, data augmentation

sub_train_x, sub_val_x, sub_train_y, sub_val_y = train_valid_split(train_x, train_y)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3), kernel_regularizer=regularizers.l2(l2=0.001)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu', activity_regularizer=regularizers.l2(0.001)))
model.add(Dense(10, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.93, beta_2=0.995, epsilon=1e-07)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Performing data augmentation
aug_datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
train_generator = aug_datagen.flow(sub_train_x, sub_train_y, batch_size=64)
validation_generator = aug_datagen.flow(sub_val_x, sub_val_y, batch_size=64)
epoch_step = int(sub_train_x.shape[0] / 64)
history = model.fit(train_generator, steps_per_epoch=epoch_step, epochs=40, validation_data=validation_generator)

results = model.evaluate(test_x, test_y)
acc_result = round(results[1], 2)
acc_result = acc_result * 100
print(f"Accuracy: {acc_result}%")

Epoch 1/40
625/625 [==============================] - 169s 265ms/step - loss: 1.6941 - accuracy: 0.3813 - val_loss: 1.3893 - val_accuracy: 0.5060
Epoch 2/40
625/625 [==============================] - 84s 134ms/step - loss: 1.3272 - accuracy: 0.5309 - val_loss: 1.1498 - val_accuracy: 0.6012
Epoch 3/40
625/625 [==============================] - 88s 141ms/step - loss: 1.1516 - accuracy: 0.5959 - val_loss: 1.0540 - val_accuracy: 0.6379
Epoch 4/40
625/625 [==============================] - 84s 134ms/step - loss: 1.0491 - accuracy: 0.6357 - val_loss: 0.9776 - val_accuracy: 0.6665
Epoch 5/40
625/625 [==============================] - 83s 133ms/step - loss: 0.9736 - accuracy: 0.6643 - val_loss: 0.9073 - val_accuracy: 0.6859
Epoch 6/40
625/625 [==============================] - 82s 132ms/step - loss: 0.9296 - accuracy: 0.6794 - val_loss: 0.8536 - val_accuracy: 0.7096
Epoch 7/40
625/625 [==============================] - 87s 140ms/step - loss: 0.8738 - accuracy: 0.6980 - val_loss: 0.8301 - val_a